# Day 17
## Part 1


In [4]:
import parse

def parse_data(s):
    r = parse.parse("""Register A: {a:d}
Register B: {b:d}
Register C: {c:d}

Program: {program}""", s)
    return r["a"], r["b"], r["c"], [int(x) for x in r["program"].split(",")]

test_data = parse_data("""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0""")
test_data

(729, 0, 0, [0, 1, 5, 4, 3, 0])

In [9]:
def combo(x, a, b, c):
    match x:
        case 0 | 1 | 2 | 3:
            return x
        case 4:
            return a
        case 5:
            return b
        case 6:
            return c

def run(a, b, c, program):
    i = 0
    while i < len(program):
        lit = program[i+1]
        com = combo(lit, a, b, c)
        match program[i]:
            case 0:
                a = a // (2 ** com)
            case 1:
                b = b ^ lit
            case 2:
                b = com % 8
            case 3:
                if a != 0:
                    i = lit - 2
            case 4:
                b = b ^ c
            case 5:
                yield com % 8
            case 6:
                b = a // (2 ** com)
            case 7:
                c = a // (2 ** com)
        i += 2

def part_1(data):
    return ",".join(str(x) for x in run(*data))

part_1(test_data)

'4,6,3,5,6,3,5,2,1,0'

In [10]:
data = parse_data(open("input").read())
part_1(data)

'1,7,6,5,1,0,5,0,7'

## Part 2

I'm going to have to think about this one.